In [1]:
import cv2
import torch
import numpy as np
import threading
import requests
from playsound import playsound

In [ ]:
#For main 1
def req(m):
    requests.get(f"http://blr1.blynk.cloud/external/api/update?token=&v0={m}")
    
def send(m):
    threading.Thread(target = req, args = (m,), daemon = True).start()

In [2]:
#For main 2
def alert():
    threading.Thread(target = playsound, args=('/Users/mayurpatowary/Desktop/Project/Tarp Helmet-Detection Project/Buzzer Sound.mp3',), daemon=True).start()

In [3]:
path = "/Users/mayurpatowary/Desktop/Project/Tarp Helmet-Detection Project/Model_weight.pt"
model = torch.hub.load('ultralytics/yolov5', 'custom', path, force_reload = True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /Users/mayurpatowary/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-11-8 Python-3.11.3 torch-2.0.1 CPU

Fusing layers... 
Model summary: 212 layers, 20861016 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [ ]:
# main 1 (with NodeMCU)
cap = cv2.VideoCapture("/Users/mayurpatowary/Desktop/Project/Capstone - Safety Helmet Compliance/TestVideos/TestVideo1.mp4")
count = 0

while True:
    ret, frame = cap.read()
    count += 1
    if count%4 != 0: 
       continue
    
    frame = cv2.resize(frame, (1020, 500))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    results = model(frame)
    st = str(results)
    output = st[st.index("1020") + 5 : st.index("Speed")]
    frame = np.squeeze(results.render())

    if 'head' in output:
        m = 1
        cv2.putText(frame, "No helmet detected!" ,(45, 36),cv2.FONT_HERSHEY_COMPLEX,(1),(0,0,255),2)
    else:
        m = 0
    
    send(m)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

m = 0
send(m)
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
#main 2 (without NodeMCU, an alert is generated in the computer itself)
cap = cv2.VideoCapture(0)
count = 0

while True:
    ret, frame = cap.read()
    count += 1
    if count%2 != 0: 
       continue
    frame = cv2.resize(frame,(1020, 500))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame)
    st = str(results)
    output = st[st.index("1020") + 5 : st.index("Speed")]
    frame = np.squeeze(results.render())
    if 'head' in output:
        cv2.putText(frame, "No helmet detected!" ,(45, 36),cv2.FONT_HERSHEY_COMPLEX,(1),(255, 0, 0),2)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)